In [ ]:
! pip install --upgrade imgaug

# Importation des modules utiles

In [ ]:
import numpy as np
import pandas as pd

import os
from glob import glob
import albumentations as albu
import cv2

# plot
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
import seaborn as sns

# manipulation image
from PIL import Image
import imageio

# data augmentation
import imgaug as ia
from imgaug import augmenters as iaa

# import segmentation maps from imgaug
from imgaug.augmentables.segmaps import SegmentationMapOnImage
import imgaug.imgaug

### Import des scripts de fonctions utiles

In [ ]:
from clouds_graph_functions import plot_image_with_masks, plot_images_and_masks
from clouds_utilities_functions import rle_to_mask, get_labels, get_mask_by_image_id, create_segmap, draw_labels, draw_segmentation_maps

# Définition du chemin des données

In [ ]:
NUAGES_PATH = '/kaggle/input/understanding_cloud_organization/'

NUAGES_TRAIN_PATH = NUAGES_PATH + 'train_images/'
NUAGES_TEST_PATH = NUAGES_PATH + 'test_images/'

# DataFrame + analyse des données

In [ ]:
train_df = pd.read_csv(NUAGES_PATH + 'train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

### Répartition des données entre entrainement et test

In [ ]:
# Chargement des noms de fichiers pour le jeu d'entrainement et test
train_fns = sorted(glob(NUAGES_TRAIN_PATH + '*.jpg'))
test_fns = sorted(glob(NUAGES_TEST_PATH + '*.jpg'))

print('Il y a {} images dans le jeu d\'entrainement.'.format(len(train_fns)))
print('Il y a {} images dans le jeu de test.'.format(len(test_fns)))

In [ ]:
# Affichage d'un camembert pour montrer la répartition des données entre jeu d'entrainement et jeu de test 
sets_size = {'Entrainement': len(train_fns), 'Test': len(test_fns)}

fig, ax = plt.subplots(figsize=(6, 6))
ax.pie(sets_size.values(), explode=(0, 0.1), labels=sets_size.keys(), autopct='%1.1f%%', shadow=True, startangle=90)
ax.axis('equal')
ax.set_title("Jeux d'entrainement et de test")

plt.show()

### Répartition des données avec masque et sans masque dans le jeu d'entrainement

In [ ]:
print('Il y a {} lignes avec des cartes de segmentation.'.format(train_df.EncodedPixels.count()))
print('Il y a {} lignes avec des cartes de segmentation vides.'.format(len(train_df) - train_df.EncodedPixels.count()))

In [ ]:
# Représentation dans un camembert de la proportion de masques vides
mask_size = {'Renseigné':train_df.EncodedPixels.count(), 'Vide': len(train_df) - train_df.EncodedPixels.count()}

fig, ax = plt.subplots(figsize=(6, 6))
ax.pie(mask_size.values(), explode=(0, 0.1), labels=mask_size.keys(), autopct='%1.1f%%', shadow=True, startangle=90)
ax.axis('equal')
ax.set_title('Masques renseignés et vides')

plt.show()

# Reorganisation du DataFrame

In [ ]:
# Exploration des labels:
# Ajout d'une colonne 'ImageId' et d'une colonne 'Label' à train_df par découpage de la colonne 'Image_label'
split_df = train_df["Image_Label"].str.split("_", n = 1, expand = True)
train_df['ImageId'] = split_df[0]
train_df['Label'] = split_df[1]
train_df['hasMask'] = ~ train_df['EncodedPixels'].isna()

In [ ]:
print(train_df.shape)
# Affichage des premières lignes du DataFrame
train_df.head()

### Répartition des différents types de nuages (labels) dans les images

In [ ]:
fish = train_df[train_df['Label'] == 'Fish'].EncodedPixels.count()
flower = train_df[train_df['Label'] == 'Flower'].EncodedPixels.count()
gravel = train_df[train_df['Label'] == 'Gravel'].EncodedPixels.count()
sugar = train_df[train_df['Label'] == 'Sugar'].EncodedPixels.count()

print('Il y a {} nuages "fish"'.format(fish))
print('Il y a {} nuages "flower"'.format(flower))
print('Il y a {} nuages "gravel"'.format(gravel))
print('Il y a {} nuages "sugar"'.format(sugar))

In [ ]:
# Fréquence des nuages suivant leur typologie
labels_size = {'Fish': fish, 'Flower': flower, 'Gravel': gravel, 'Sugar': sugar}

fig, ax = plt.subplots(figsize=(6, 6))
ax.pie(labels_size.values(), labels=labels_size.keys(), autopct='%1.1f%%', shadow=True, startangle=90)
ax.axis('equal')
ax.set_title('Catégories de nuages')

plt.show()

## Exploration du nombre moyen de labels par image

In [ ]:
labels_per_image = train_df.groupby('ImageId')['EncodedPixels'].count()
print('Le nombre moyen de labels par image est de {}'.format(labels_per_image.mean()))

## Répartition des labels par image

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.hist(labels_per_image)
ax.set_title('Histogramme du nombre de labels par image')
plt.show()

## Exploration des corrélations entre les différents types de nuages

In [ ]:
train_df = train_df.fillna('-1')

In [ ]:
def dummy_var(label):
    values = []
    df_temp = train_df[train_df["Label"]==label]
    df_temp["Dummy"] = df_temp["EncodedPixels"].apply(lambda x: 1 if x!= '-1' else 0)
    return list(df_temp["Dummy"])

df_images = pd.DataFrame()
df_images["ImageId"] = train_df["ImageId"].unique()
for i in train_df["Label"].unique():
    df_images[i] = dummy_var(i)


In [ ]:
df_images.head()

In [ ]:
sns.set(font_scale=1)
sns.set(rc={'figure.figsize':(15,15)})
hm=sns.heatmap(df_images.corr(), cbar = True, annot=True, square = True, fmt = '.2f',
              yticklabels = ['Fish', 'Flower', 'Gravel', 'Sugar'], 
            xticklabels = ['Fish', 'Flower', 'Gravel', 'Sugar']).set_title('Heatmap de correlation des nuages')

fig = hm.get_figure()

Il n'y a pas de corrélation forte entre les différents types de nuages.
Plus forte corrélation = 24%, entre les nuages de type 'Gravel' et les nuages de type 'Flower'.

# Exploration des images:

In [ ]:
# Fonction pour afficher une grille d'images et leurs labels
def plot_training_images(width = 5, height = 2):
    """
    Function to plot grid with several examples of cloud images from train set.
    INPUT:
        width - number of images per row
        height - number of rows

    OUTPUT: None
    """
    
    # get a list of images from training set
    images = sorted(glob(NUAGES_TRAIN_PATH + '*.jpg'))
    
    fig, axs = plt.subplots(height, width, figsize=(width * 3, height * 3))
    
    # create a list of random indices 
    rnd_indices = [np.random.choice(range(0, len(images))) for i in range(height * width)]
    
    for im in range(0, height * width):
        # open image with a random index
        image = Image.open(images[rnd_indices[im]])
        
        i = im // width
        j = im % width
        
        # plot the image
        axs[i,j].imshow(image) #plot the data
        axs[i,j].axis('off')
        axs[i,j].set_title(get_labels(train_df, images[rnd_indices[im]].split('/')[-1]))

    # set suptitle
    plt.suptitle("Echantillon d'images du jeu d'entrainement")
    plt.show()

In [ ]:
plot_training_images()

## Affichage d'une image avec ses masques

In [ ]:
plot_image_with_masks(train_df, NUAGES_TRAIN_PATH + "0011165.jpg")

## Affichage de plusieurs images avec leurs masques

In [ ]:
plot_images_and_masks(train_df, NUAGES_TRAIN_PATH)